In [61]:
from sbhandler import *
import pickle

# load_matlab_file function Testing
file_name = 'consensus1.mat'
variable_of_interest = 'align3'
h5py_object, data = load_matlab_file(file_name, variable_name=variable_of_interest)

# DNA_seqs can be used to grab each raw_seqblock
DNA_seqs = DNA_SeqBlocks(h5py_object=h5py_object, data=data) 

FILE_SIZE = DNA_seqs.size #Constant

# Load 'em in 
# df = load(open('pickle/v2BarcodesTruelen.pickle', 'rb'))
# BARCODES = df.barcode
# TRUELENS = df.true_len

bfp_list = pickle.load( open('pickle/v1rfp.pickle', 'rb'))
rfp_list = pickle.load( open('pickle/v1bfp.pickle', 'rb'))

# tracking = load( open('pickle/v2TrackingwRFP.pickle', 'rb'))


# Weighted Voting

In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from fastai.tabular import *
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import time


## Data Preprocessing
- Features: 

In [ ]:
# from fastbook import one_hot, one_hot_decode
toBeX = []
toBeY = []
conversion = dict(zip([65, 67, 71, 84, 45], [0, 1, 2, 3, 4]))
# print(conversion

def useableX(dfslice, idx):
    temp_ct = dict(zip([65, 67, 71, 84, 45], [0, 0, 0, 0, 0]))
    temp_q = dict(zip([65, 67, 71, 84, 45], [0, 0, 0, 0, 0]))
    
    cov = int( (len(dfslice.columns) - 4) / 2)
    for read in range(cov): # Easy. Just adjust the count
        nuc_read = dfslice[f'read{read+1}'].loc[idx]
        temp_ct[nuc_read] += 1
        nuc_qual = dfslice[f'q{read+1}'].loc[idx]
        if nuc_qual > temp_q[nuc_read]: # Currently using keep the max. Is there some way to combine them?
            # Note: my issue is that if I combine a 73 and a 105 for T. It's not better than a single A for 126 is it?
            temp_q[nuc_read] = int(nuc_qual)
    counts = torch.tensor(list(temp_ct.values()))
    qscores = torch.tensor(list(temp_q.values()))
    return torch.concat([counts, qscores])

# Arbitrarily grabs slices (no regard for whether they have yyy or z conflicts)
# for bfp in bfp_list[202:300]:

#     print(bfp.index.name)
#     for idx in range(len(bfp.index)):
#         true_nuc = bfp['target'].iloc[idx] 
#         toBeY.append(conversion[true_nuc])
#         toBeX.append(useableX(bfp, idx))
# len(toBeX), len(toBeY)

for bfp in bfp_list:
    print(bfp.index.name)
    if bfp.empty: continue
    focus = bfp[bfp['changes'] ==122]
    if focus.empty: continue
    for idx in list(focus.index):
        true_nuc = bfp['target'].loc[idx] 
        toBeY.append(conversion[true_nuc])
        toBeX.append(useableX(bfp, idx))
len(toBeX), len(toBeY)



In [88]:
classes = ['A', 'C', 'G', 'T', '-']
    
raw_train_x = torch.stack(toBeX).float()
raw_train_y = torch.stack(list(map(torch.tensor, toBeY))).long()

train_x = raw_train_x/126
train_y = raw_train_y

In [ ]:
import random

def rfpX(dfslice, idx, cov):
    temp_ct = dict(zip([65, 67, 71, 84, 45], [0, 0, 0, 0, 0]))
    temp_q  = dict(zip([65, 67, 71, 84, 45], [0, 0, 0, 0, 0]))

    sample = [r for r in range(cov)] # Easy. Just adjust the count
    to_use = (random.sample(sample, round(cov/2)))
    print(to_use)
    for read in to_use:
        nuc_read = dfslice[f'read{read+1}'].loc[idx]
        temp_ct[nuc_read] += 1
        nuc_qual = dfslice[f'q{read+1}'].loc[idx]
        if nuc_qual > temp_q[nuc_read]: # Currently using keep the max. Is there some way to combine them?
            # Note: my issue is that if I combine a 73 and a 105 for T. It's not better than a single A for 126 is it?
            temp_q[nuc_read] = int(nuc_qual)
    counts = torch.tensor(list(temp_ct.values()))
    qscores = torch.tensor(list(temp_q.values()))
    return torch.concat([counts, qscores])


for rfp in rfp_list:
    if rfp.empty: continue
    
    cov = int( (len(rfp.columns)-4)/2)
    if cov <= 3: continue 
    
    focus = rfp[rfp['changes'] ==122]
    if focus.empty: continue
    
    for idx in list(focus.index):
        print(idx)
        true_nuc = rfp['contig'].loc[idx] 
        toBeY.append(conversion[true_nuc])
        toBeX.append(rfpX(rfp, idx, cov))
len(toBeX), len(toBeY)

In [97]:
    
raw_valid_x = torch.stack(toBeX).float()
raw_valid_y = torch.stack(list(map(torch.tensor, toBeY))).long()

valid_x = raw_valid_x/126
valid_x = raw_valid_x/126
valid_x = valid_x[-20_000:]
valid_y = valid_y[-20_000:]

torch.Size([70804, 10]) torch.Size([44816, 10])


In [98]:
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

(torch.Size([95620, 10]),
 torch.Size([95620]),
 torch.Size([20000, 10]),
 torch.Size([20000]))

## Subject to Change: I/O of LogReg Model
- Input: 10 predictors (Act, Cct, Gct, Tct, Ict, errA, errC, errG, errT, errI)
- Parameters: TBD -> use model selection techniques
- Output: 5 classes (A, C, G, T, I)



In [99]:
class SNPDataset(torch.utils.data.Dataset):
    
    def __init__(self, X, y):
        super().__init__();
        self.X, self.y = X,y
        self.len = X.shape[0] 
    def __len__(self): return self.len
    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]       

In [100]:
train_ds = SNPDataset(train_x, train_y)
valid_ds = SNPDataset(valid_x, valid_y)

batch_size=200
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

In [101]:
class LogReg(nn.Module):
    def __init__(self):
        super(LogReg, self).__init__();
        # self.input_dim, self.output_dim, self.hidden_dim = in_dim, out_dim, hidden_dim
        # self.linear = nn.Linear(in_features=input_dim, out_features=hidden_dim, bias=True)
        
        # Partial first layer
        # self.linearA = nn.Linear(in_features=2, out_features=1, bias=True)
        # self.linearC = nn.Linear(in_features=2, out_features=1, bias=True)
        # self.linearG = nn.Linear(in_features=2, out_features=1, bias=True)
        # self.linearT = nn.Linear(in_features=2, out_features=1, bias=True)
        # self.linearI = nn.Linear(in_features=2, out_features=1, bias=True)   
        self.linear1 = nn.Linear(in_features=10, out_features=20, bias=True)
        
        # Dense second layer (final)
        self.linear2 = nn.Linear(in_features=20, out_features=5, bias=True)
        
        # the activation function used
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # outA = self.linearA(x[:,[0,5]]) #Act and AqualityTotal
        # outC = self.linearC(x[:,[1,6]])
        # outG = self.linearG(x[:,[2,7]])
        # outT = self.linearT(x[:,[3,8]])
        # outI = self.linearI(x[:,[4,9]])
        
        
        # for out in [outA, outC, outG, outT, outI]:
        #     # print(out)
        #     out = self.relu(out)
        # logits = torch.hstack([outA, outC, outG, outT, outI]).view(-1, 5) # Possible point of error. Confirm that .stack().view() are actually giving columns outA, outC, outG, outT, outI, etc.
        # print(logits)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [102]:
logreg = LogReg()
logreg = logreg.to('cuda')
loss_func = nn.CrossEntropyLoss()


In [104]:
lr=1e-4
optim = torch.optim.SGD(params=logreg.parameters(), lr=lr)

n_epochs = 35

In [112]:
# weights_prior = logreg.linear.weight.detach().numpy().copy()

logreg.train()

for epoch in range(1, n_epochs+1):
    # Observations
    n_obs = 0
    loss_sum = 0
    n_right = 0
    
    for xb, yb in train_dl:
        xb = xb.to('cuda')
        yb = yb.to('cuda')
        optim.zero_grad() # Reset gradient
        
        # Forward pass.
        logit = logreg.forward(xb) # Get logit_batch
        loss = loss_func(logit, yb) # Calculate loss.
        
        # Update. Learn.
        loss.backward()
        optim.step()
        
        # Update observation metrics
        n_obs += len(yb)
        loss_sum += loss.item() * n_obs
        n_right += (logit.argmax(1) == yb.long()).float().sum().item()
    
    loss_train = loss_sum / n_obs
    acc_train = n_right / n_obs

    with torch.no_grad():
        # Observations
        n_obs = 0
        loss_sum = 0
        n_right = 0
        
        for xb, yb in valid_dl:
            xb = xb.to('cuda')
            yb = yb.to('cuda')
            # Forward pass.
            logit = logreg.forward(xb) # Get logit_batch
            loss = loss_func(logit, yb) # Calculate loss.
            
            # Update observation metrics
            n_obs += len(yb)
            loss_sum += loss.item() * n_obs
            n_right += (logit.argmax(1) == yb.long()).float().sum().item()
        
    loss_valid = loss_sum / n_obs
    acc_valid = n_right / n_obs
    
    disp_prog_freq = 1
    if ((epoch % disp_prog_freq == 0) | (epoch == 0) | (epoch == n_epochs)): # Show for first epoch, last epoch, and based on desired frequency
        print(f'  E{(epoch):02d} | train loss: {loss_train:.4f} | train acc: {acc_train:.4f} | valid loss: {loss_valid:.4f} | valid acc: {acc_valid:.4f}')
        
print('\nModel training complete.\n')

# weights_after = logreg.linear.weight.detach().numpy().copy()
        
    

  E01 | train loss: 142.1464 | train acc: 0.8580 | valid loss: 25.7437 | valid acc: 0.8887
  E02 | train loss: 141.3671 | train acc: 0.8586 | valid loss: 25.5715 | valid acc: 0.8891
  E03 | train loss: 140.5893 | train acc: 0.8593 | valid loss: 25.3950 | valid acc: 0.8893
  E04 | train loss: 139.8630 | train acc: 0.8599 | valid loss: 25.2246 | valid acc: 0.8895
  E05 | train loss: 139.5004 | train acc: 0.8606 | valid loss: 25.0554 | valid acc: 0.8897
  E06 | train loss: 138.4573 | train acc: 0.8608 | valid loss: 24.8895 | valid acc: 0.8898
  E07 | train loss: 137.6518 | train acc: 0.8613 | valid loss: 24.7270 | valid acc: 0.8901
  E08 | train loss: 137.0449 | train acc: 0.8618 | valid loss: 24.5612 | valid acc: 0.8904
  E09 | train loss: 135.9079 | train acc: 0.8622 | valid loss: 24.3991 | valid acc: 0.8909
  E10 | train loss: 135.5674 | train acc: 0.8627 | valid loss: 24.2408 | valid acc: 0.8914
  E11 | train loss: 135.0170 | train acc: 0.8631 | valid loss: 24.0806 | valid acc: 0.8918

## Validation

In [110]:
logreg.eval()

with torch.no_grad():
     # Observations
    n_obs = 0
    loss_sum = 0
    n_right = 0
    
    for xb, yb in valid_dl:
        xb = xb.to('cuda')
        yb = yb.to('cuda')
        # Forward pass.
        logit = logreg.forward(xb) # Get logit_batch
        loss = loss_func(logit, yb) # Calculate loss.
        
        # Update observation metrics
        n_obs += len(yb)
        loss_sum += loss.item() * n_obs
        n_right += (logit.argmax(1) == yb.long()).float().sum().item()
    
    loss_train = loss_sum / n_obs
    acc_train = n_right / n_obs
    
    disp_prog_freq = 1
    if ((epoch % disp_prog_freq == 0) | (epoch == 0) | (epoch == n_epochs)): # Show for first epoch, last epoch, and based on desired frequency
        print(f'  valid loss: {loss_train:.4f} | valid acc: {acc_train:.4f}')


  valid loss: 33.2568 | valid acc: 0.8718

Model validation complete.

